In [1]:
from pyMSOO.MFEA.model import MaTGA
from pyMSOO.utils.Crossover import *
from pyMSOO.utils.Mutation import *
from pyMSOO.utils.Selection import *
from pyMSOO.MFEA.benchmark.continous import *
from pyMSOO.utils.MultiRun.RunMultiTime import * 

In [2]:
import numpy as np
import random
from functools import reduce
import time
import copy

from pyMSOO.MFEA.model import AbstractModel
from pyMSOO.utils import Crossover, Mutation, Selection
from pyMSOO.utils.EA import *
from pyMSOO.utils.numba_utils import numba_randomchoice

class model(AbstractModel.model):
    def compile(self, 
        IndClass: Type[Individual],
        tasks: List[AbstractTask], 
        crossover: Crossover.SBX_Crossover, mutation: Mutation.PolynomialMutation, selection: Selection.ElitismSelection, 
        *args, **kwargs):
        super().compile(IndClass, tasks, crossover, mutation, selection, *args, **kwargs)
    
    def fit(self, nb_generations, B = 0.25, H = 0.5, nb_inds_each_task = 100, evaluate_initial_skillFactor = True, *args, **kwargs) -> List[Individual]:
        super().fit(*args, **kwargs)

        # initialize population
        self.population = Population(
            self.IndClass,
            nb_inds_tasks = [nb_inds_each_task] * len(self.tasks), 
            dim = self.dim_uss,
            list_tasks= self.tasks,
            evaluate_initial_skillFactor = evaluate_initial_skillFactor
        )
        self.B = B
        self.H = H

        self.M = np.ones((len(self.tasks), len(self.tasks)))
        self.N = np.ones((len(self.tasks), len(self.tasks)))
        self.C = np.ones((len(self.tasks), len(self.tasks)))
        self.O = np.ones((len(self.tasks), len(self.tasks)))
        self.P = np.ones((len(self.tasks), len(self.tasks)))
        self.A = np.ones((len(self.tasks), len(self.tasks)))
        self.R = np.ones((len(self.tasks), len(self.tasks)))
        
        # save history
        self.history_cost.append([ind.fcost for ind in self.population.get_solves()])
        
        self.render_process(0, ['Cost'], [self.history_cost[-1]], use_sys= True)

        for epoch in range(nb_generations):

            offsprings = Population(
                self.IndClass,
                nb_inds_tasks = [0] * len(self.tasks), 
                dim = self.dim_uss,
                list_tasks= self.tasks,
            )

            copy_offsprings = Population(
                self.IndClass,
                nb_inds_tasks = [0] * len(self.tasks), 
                dim = self.dim_uss,
                list_tasks= self.tasks,
            )

            subPop = [0] * len(self.tasks)
            other_task = [0] * len(self.tasks)

            for i in range(len(self.tasks)):
                offs = Population(
                    self.IndClass,
                    nb_inds_tasks = [0] * len(self.tasks), 
                    dim = self.dim_uss,
                    list_tasks= self.tasks,
                )
                while len(offs) < nb_inds_each_task:
                    # choose parent 
                    pa, pb = self.population[i].__getRandomItems__(2)
                    # intra / inter crossover
                    oa, ob = self.crossover(pa, pb, i, i)
                    # mutate
                    oa = self.mutation(oa, return_newInd= True)
                    oa.skill_factor = pa.skill_factor

                    ob = self.mutation(ob, return_newInd= True)    
                    ob.skill_factor = pb.skill_factor
                    
                    offs.__addIndividual__(oa)
                    offs.__addIndividual__(ob)

                offsprings += offs
                copy_offsprings += offs
            
            # print(len(copy_offsprings[1]))
            for i, t in enumerate(self.tasks):

                self.R[i][i] = -1
                R_i = max(self.R[i])
                if random.random() < R_i:
                    task_j = np.argmax(self.R[i])
                    S_i = min(max(int(R_i * nb_inds_each_task), 0), nb_inds_each_task)
                    other_task[i] = task_j

                    # print(len(offsprings[task_j]), len(copy_offsprings[task_j]))
                    # print(copy_offsprings[task_j][0: S_i])
                    
                    subPop[i] = [self.IndClass(off.genes, skill_factor=i, fcost=t(off.genes)) for off in copy_offsprings[task_j][0: S_i]]
                    offsprings[i][nb_inds_each_task - S_i: nb_inds_each_task] = subPop[i]
                    
                    # print(i, len(subPop[i]), len(copy_offsprings[task_j]), S_i, task_j)
                    # print(subPop[i][0].skill_factor)
            # merge and update rank
            self.population = self.population + offsprings
            self.population.update_rank()

            # selection
            self.selection(self.population, [nb_inds_each_task] * len(self.tasks))

            # update symbiosis and rate
            self.update_symbiosis(subPop, other_task)
            self.update_rate()

            # update operators
            self.crossover.update(population = self.population)
            self.mutation.update(population = self.population)

            # save history
            self.history_cost.append([ind.fcost for ind in self.population.get_solves()])

            #print
            self.render_process((epoch+1)/nb_generations, ['Cost'], [self.history_cost[-1]], use_sys= True)
        
        print('\nEND!')

        #solve 
        self.last_pop = self.population
        return self.last_pop.get_solves()

    def update_rate(self):
        for t in range(len(self.tasks)):
            T_pos = self.M[t] + self.O[t] + self.P[t]
            T_neg = self.A[t] + self.C[t]
            T_neu = self.M[t]
            self.R[t] = T_pos/(T_pos + T_neg + T_neu)
            self.R[t][t] = -1

    def update_symbiosis(self, subpop: List[Individual], other_task):
        for i in range(len(self.tasks)):
            if isinstance(subpop[i], list):
                for o in subpop[i]:
                    j = other_task[i]
                    if i != j:
                        r_i = self.rank(o, i)
                        r_j = self.rank(o, j)
                        if (self.isBenefit(r_i) and self.isBenefit(r_j)) :
                            self.M[i][j] += 1
                        elif (self.isNeural(r_i) and self.isNeural(r_j)):
                            self.N[i][j] += 1
                        elif (self.isHarmful(r_i) and self.isHarmful(r_j)):
                            self.C[i][j] += 1
                        elif (self.isBenefit(r_i) and self.isNeural(r_j)):
                            self.O[i][j] += 1
                        elif (self.isBenefit(r_i) and self.isHarmful(r_j)): 
                            self.P[i][j] += 1
                        elif (self.isNeural(r_i) and self.isHarmful(r_j)):
                            self.A[i][j] += 1

    def rank(self, off: Individual, task):
        fitness = self.tasks[task](off)
        # print(self.population[task].getFitness())
        rank = np.searchsorted(self.population[task].getFitness(), fitness)
        return rank/len(self.population[task])
    
    def isBenefit(self, r):
        if (r <= self.B):
            return True
        return False

    def isHarmful(self, r) :
        if (r > self.H) :
            return True
        return False

    def isNeural(self, r) :
        return (r > self.B and r <= self.H)

In [3]:
# tasks, IndClass = CEC17_benchmark.get_2tasks_benchmark(1)
# tasks, IndClass = WCCI22_benchmark.get_complex_benchmark(1)
tasks, IndClass = CEC17_benchmark.get_10tasks_benchmark()

In [4]:
baseModel = model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection()
)
solve = baseModel.fit(
    nb_generations = 1000,nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

Seed: None -- Time: 01m 21.12s  100 % [====================>]  Cost: 1.95E-06  3.35E-07  1.88E-10  0.00E+00  2.44E-04  1.72E-04  1.50E-03  6.38E-04  2.40E-08  2.56E+01  ,  
END!


In [5]:
baseModel.R

array([[-1.        ,  0.04545455,  0.04545455,  0.04411765,  0.04511278,
         0.04347826,  0.04347826,  0.04347826,  0.04477612,  0.04477612],
       [ 0.06293706, -1.        ,  0.05769231,  0.05660377,  0.06024096,
         0.11681914,  0.06      ,  0.05357143,  0.0625    ,  0.05769231],
       [ 0.05263158,  0.05357143, -1.        ,  0.05172414,  0.05263158,
         0.05263158,  0.16790231,  0.05555556,  0.05172414,  0.05263158],
       [ 0.04494382,  0.04347826,  0.04347826, -1.        ,  0.04545455,
         0.04545455,  0.04545455,  0.04347826,  0.04477612,  0.04411765],
       [ 0.15480427,  0.10344828,  0.10204082,  0.05882353, -1.        ,
         0.08163265,  0.05357143,  0.05555556,  0.07894737,  0.08108108],
       [ 0.11111111,  0.11685995,  0.1038961 ,  0.11538462,  0.05882353,
        -1.        ,  0.10227273,  0.06521739,  0.075     ,  0.0625    ],
       [ 0.04411765,  0.04615385,  0.04705882,  0.04924242,  0.04761905,
         0.04918033, -1.        ,  0.046875  